# Python 下載XML檔案與解析


* 了解 xml 檔案格式與內容
* 能夠利用套件存取 xml 格式的檔案


## 作業目標

* 比較一下範例檔案中的「File I/O」與「xmltodict」讀出來的內容有什麼差異

* 根據範例檔案的結果：
    1. 請問高雄市有多少地區有溫度資料？
    2. 請取出每一個地區所記錄的第一個時間點跟溫度
    3. 請取出第一個地區所記錄的每一個時間點跟溫度

In [1]:
# My Code

# Download File
import urllib.request
import zipfile

downloadURL = "http://opendata.cwb.gov.tw/govdownload?dataid=F-D0047-093&authorizationkey=rdec-key-123-45678-011121314"
urllib.request.urlretrieve(downloadURL, "./data/Day003_Data/example.zip")
file = zipfile.ZipFile('./data/Day003_Data/example.zip')
file.extractall('./data/Day003_Data')

### 比較一下範例檔案中的「File I/O」與「xmltodict」讀出來的內容有什麼差異


### File I/O
-  File I/O 開出來的檔案是str類型，須以slice的方式找到特定資訊

In [34]:
# My Code

# Read the File
content = open("./data/Day003_Data/TAIWAN_Weekday_CH.xml", "r", encoding="utf-8")
xml = content.read()
content.close()

print(type(xml),'\n')
print(xml[:1000])

<class 'str'> 

<?xml version="1.0" encoding="utf-8"?>
<cwbopendata xmlns="urn:cwb:gov:tw:cwbcommon:0.1">
  <identifier>21120e6c-69ef-4297-bd06-1438bb8d59a7</identifier>
  <sender>weather@cwb.gov.tw</sender>
  <sent>2019-12-04T12:21:10+08:00</sent>
  <status>Actual</status>
  <scope>Public</scope>
  <msgType>Issue</msgType>
  <dataid>D0047-091</dataid>
  <source>MFC</source>
  <dataset>
    <datasetInfo>
      <datasetDescription>臺灣各縣市鄉鎮未來1週逐12小時天氣預報</datasetDescription>
      <datasetLanguage>zh-TW</datasetLanguage>
      <issueTime>2019-12-04T11:00:00+08:00</issueTime>
      <validTime>
        <startTime>2019-12-04T12:00:00+08:00</startTime>
        <endTime>2019-12-12T11:00:00+08:00</endTime>
      </validTime>
      <update>2019-12-04T12:21:10+08:00</update>
    </datasetInfo>
    <locations>
      <locationsName>台灣</locationsName>
      <location>
        <locationName>連江縣</locationName>
        <geocode>09007</geocode>
        <lat>26.154204</lat>
        <lon>119.929303</lon>
 

In [30]:
print(xml[392:748])

      <datasetDescription>臺灣各縣市鄉鎮未來1週逐12小時天氣預報</datasetDescription>
      <datasetLanguage>zh-TW</datasetLanguage>
      <issueTime>2019-12-04T11:00:00+08:00</issueTime>
      <validTime>
        <startTime>2019-12-04T12:00:00+08:00</startTime>
        <endTime>2019-12-12T11:00:00+08:00</endTime>
      </validTime>
      <update>2019-12-04T12:21:10+08:00


### xmltodict
-  xmltodict 開出來的檔案是dict類型，可以鍵(key)找到特定資訊

In [35]:
# My Code

import xmltodict

print(type(xmltodict.parse(xml)),'\n')

dict(xmltodict.parse(xml)['cwbopendata']['dataset']['datasetInfo'])

<class 'collections.OrderedDict'> 



{'datasetDescription': '臺灣各縣市鄉鎮未來1週逐12小時天氣預報',
 'datasetLanguage': 'zh-TW',
 'issueTime': '2019-12-04T11:00:00+08:00',
 'validTime': OrderedDict([('startTime', '2019-12-04T12:00:00+08:00'),
              ('endTime', '2019-12-12T11:00:00+08:00')]),
 'update': '2019-12-04T12:21:10+08:00'}

### 根據範例檔案的結果：

1. 請問高雄市有多少地區有溫度資料？
2. 請取出每一個地區所記錄的第一個時間點跟溫度
3. 請取出第一個地區所記錄的每一個時間點跟溫度

In [15]:
# 1. 請問高雄市有多少地區有溫度資料？

# My Code
import os
files = os.listdir('./data/Day003_Data')

for file in files:
    
    if 'CH' in file:
        
        with open("./data/Day003_Data/" + file, encoding='utf-8') as file:
            data = dict(xmltodict.parse(file.read()))
            city = data['cwbopendata']['dataset']['locations']['locationsName']
            
            if '高雄市' in city:
                areas = data['cwbopendata']['dataset']['locations']['location']
                print('高雄市共有 ' + str(len(areas)) + " 區有溫度資料")   # area[x] --> different x, different areas
                break

高雄市共有 38 區有溫度資料


In [24]:
# 2. 請取出每一個地區所記錄的第一個時間點跟溫度

# My Code
for area in areas:
    areaName = area['locationName'] + ' : '
    
    for item in area['weatherElement']:
        if (item['description'] == '溫度'):
            data = item['time'][0]            
            time = data['dataTime']
            value = data['elementValue']['value']
            measures = data['elementValue']['measures']
            print(areaName + str(time) + "  溫度為 " + str(value) + "("+measures+")")

鹽埕區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
鼓山區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
左營區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
楠梓區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
三民區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
新興區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
前金區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
苓雅區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
前鎮區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
旗津區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
小港區 : 2019-12-04T12:00:00+08:00  溫度為 23(攝氏度)
鳳山區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
林園區 : 2019-12-04T12:00:00+08:00  溫度為 23(攝氏度)
大寮區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
大樹區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
大社區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
仁武區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
鳥松區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
岡山區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
橋頭區 : 2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
燕巢區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
田寮區 : 2019-12-04T12:00:00+08:00  溫度為 21(攝氏度)
阿蓮區 : 2019

In [27]:
# 3. 請取出第一個地區所記錄的每一個時間點跟溫度

# My Code
firstAreaName = areas[0]
print(firstAreaName['locationName'] + ' : ')

for item in firstArea['weatherElement']:
    if (item['description'] == '溫度'):
        allData = item['time']
        
        for data in allData:
            dataTime = data['dataTime']
            value = data['elementValue']['value']
            measures = data['elementValue']['measures']
            print(str(dataTime) + "  溫度為 " + str(value) + "("+measures+")")

鹽埕區 : 
2019-12-04T12:00:00+08:00  溫度為 22(攝氏度)
2019-12-04T15:00:00+08:00  溫度為 22(攝氏度)
2019-12-04T18:00:00+08:00  溫度為 21(攝氏度)
2019-12-04T21:00:00+08:00  溫度為 19(攝氏度)
2019-12-05T00:00:00+08:00  溫度為 18(攝氏度)
2019-12-05T03:00:00+08:00  溫度為 17(攝氏度)
2019-12-05T06:00:00+08:00  溫度為 17(攝氏度)
2019-12-05T09:00:00+08:00  溫度為 18(攝氏度)
2019-12-05T12:00:00+08:00  溫度為 21(攝氏度)
2019-12-05T15:00:00+08:00  溫度為 22(攝氏度)
2019-12-05T18:00:00+08:00  溫度為 20(攝氏度)
2019-12-05T21:00:00+08:00  溫度為 19(攝氏度)
2019-12-06T00:00:00+08:00  溫度為 18(攝氏度)
2019-12-06T03:00:00+08:00  溫度為 17(攝氏度)
2019-12-06T06:00:00+08:00  溫度為 17(攝氏度)
2019-12-06T09:00:00+08:00  溫度為 17(攝氏度)
2019-12-06T12:00:00+08:00  溫度為 19(攝氏度)
2019-12-06T15:00:00+08:00  溫度為 20(攝氏度)
2019-12-06T18:00:00+08:00  溫度為 18(攝氏度)
2019-12-06T21:00:00+08:00  溫度為 17(攝氏度)
2019-12-07T00:00:00+08:00  溫度為 16(攝氏度)
2019-12-07T03:00:00+08:00  溫度為 16(攝氏度)
2019-12-07T06:00:00+08:00  溫度為 16(攝氏度)
2019-12-07T09:00:00+08:00  溫度為 17(攝氏度)
